<a href="https://colab.research.google.com/github/szyrek/sentiment_anal/blob/main/Sentiments%2BScrap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install dependencies

!pip install -q transformers sumy feedparser feedsearch bertopic

!python -c "import nltk; nltk.download('punkt')"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.4/143.4 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 16.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.2/88.2 kB 7.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 31.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
# Import external definitions

from transformers import pipeline
from sumy.summarizers.lex_rank import LexRankSummarizer
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.text_rank import TextRankSummarizer
from time import time, sleep
from feedsearch import search
from feedparser import parse
import torch


from transformers import AutoTokenizer, AutoModelWithLMHead
from bertopic import BERTopic


In [ ]:
tokenizer = AutoTokenizer.from_pretrained('t5-base')
model = AutoModelWithLMHead.from_pretrained('t5-base', return_dict=True)

/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/auto/modeling_auto.py:1423: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language

In [ ]:
# Config

logging = True
timing = True

summarizer = LexRankSummarizer()
tokenizer = Tokenizer("english")
pipe = pipeline("sentiment-analysis")
#feeds = search('pap.pl')
feeds = [type('obj', (object,), {"url" : "https://www.pap.pl/rss.xml"})]


No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [ ]:
# Internal definitions

# logging
def log(text):
  if logging is True:
    print(text)

def log_time(activity_name, start_time):
  if timing is True:
    print(activity_name + " took %s seconds" % (time() - start_time))

# main features
def get_articles():
  urls = [f.url for f in feeds]

  print(urls)

  if len(urls) > 0:
    news_feed = parse(urls[0])
    entries = news_feed["entries"]
    return entries

def summarize_data(text_data, summarizer, tokenizer):
  if summarizer is None or tokenizer is None:
    log("using raw data, since summarizer or tokenizer not set")
    return text_data

  if timing is True:
    start_time = time()

  summarized_data = []
  for text in text_data:
      parser = PlaintextParser.from_string(text, tokenizer)
      summary = summarizer(parser.document, 2)

      text_summary = ""
      for sentence in summary:
          text_summary+=str(sentence)

      summarized_data.append(text_summary)
  log_time("data summarization", start_time)
  return summarized_data

def analyse_sentiment(text_data, sentiment_pipeline):
  if timing is True:
    start_time = time()
  results = sentiment_pipeline(text_data)
  log_time("data analysis", start_time)
  return results


In [ ]:
# Data definition

text_data = [
"""
Bitcoin had a great start to 2023, reversing the downward trend seen throughout the end of 2022. From $16,000 on 1 January, it has risen 63% to $27,152 so far this year. Last month, the BTC price even exceeded $30,000 on hopes that the Fed’s hiking cycle was nearing completion and that inflation was calming down.
However, recent weeks have seen bitcoin retrace its steps. We think the main reason for this is a deteriorating macro backdrop. The labour market remains very tight. Inflation remains way above target (April CPI showed little progress on disinflation, with core goods rising 0.6% due to a 4.4% increase in used care prices). And Fed Chair Jerome Powell recently said ‘‘the process of getting inflation back down to 2 percent has a long way to go.’
Currently, the markets are assigning merely a 5% chance of a 25 basis point increase by the Federal Open Market Committee (FOMC) in June. From our perspective, the markets are underpricing the likelihood of such a hike. Considering the ongoing inflation and growth dynamics, it is probable that a 25 basis point increase will occur in June (unless a debt ceiling crisis arises), which would have a bearish effect on bitcoin.
Should you buy BTC now? The quick answer is ‘probably not’. The macro backdrop for bitcoin is bearish. We analyse various on-chain/flow metrics for bitcoin, which are neutral. So, overall, we are slightly bearish. Therefore, if you have a two-to-four-week horizon, now may not be a good time to buy bitcoin.
""",
"""
On July 24, 2023, Coinglass data revealed that over $41 million worth of Bitcoin long and short leveraged positions were liquidated as the BTC prices unexpectedly crashed below a consolidation level of around $29,500, shrinking by over 4%. The drop below the primary support is heaping pressure on the coin.
It may draw more selling pressure in upcoming sessions, pushing prices toward immediate reaction lines, the next being at about $28,300.
Bitcoin Drops, Over $41 Million Of Longs Liquidated
In crypto trading, liquidation happens the facilitating exchange, for example, OKX or Binance, forcibly takes over the collateral securing the leverage position whenever prices move against the trader’s prognosis. In this case, the recent liquidation was triggered by the rapid sell-off in Bitcoin, leading to a more than 4% price decline within a few hours during the New York Session on July 24.
""",
"""
Bitcoin (BTC) was on the brink of falling outside of the $29,000 mark on Monday, as traders began to prepare for a big week of economic data.
Following Sunday’s high of $30,330.64, BTC/USD slipped to an intraday low of $29,068.48 earlier in the session.
This drop has resulted in bitcoin falling to its weakest point since June 21, which is the last time price was below $29,000.
Overall, bitcoin is over 3% lower than at the same time last week, with today’s sell-off coming when the relative strength index (RSI) fell below a floor of 45.00.
Currently, price strength is tracking at 42.93, with the next visible resistance point at the 41.00 mark.
Should the index reach this level, it is highly likely that BTC will be trading around the $28,800 level.
""",
"""
Investing in Bitcoin is a good idea in 2023. Since its launch in 2009, the price of Bitcoin has grown exponentially, generating a great deal of investor interest.
Many feel that bitcoin is here to stay and it makes a good investment option for those looking to diversify their portfolio.
But is true? Is it really safe to invest in Bitcoin this year? Let us find out below. WHAT ARE THE BENEFITS OF INVESTING IN BITCOIN NOW?
There’s no doubt that Bitcoin is an exciting and very promising investment. The price of bitcoin could rise substantially in the future, so
it’s a wise investment right now. The tax benefits of holding Bitcoin are also significant.
""",
"""
Investing in bitcoin now is a brilliant idea since it’s a cryptocurrency with much potential. There are many benefits, like greater security and lower
transaction fees, which can help diversify your portfolio. A lot of financial experts say it’s an excellent long-term investment. Your financial situation
and risk tolerance will determine whether you buy bitcoin in January 2023. If you’re looking for a long-term investment, consider buying Bitcoin at the start
of 2023. If you’re looking for a short-term investment, now is the best time to get in on Bitcoin’s price rise.
""",
"""
Bitcoin fell on Tuesday morning in Asia along with most other leading cryptocurrencies as investors took a risk-averse approach on speculation the U.S. regulator
will appeal a decision in the Ripple court case and the view the recent price gain on hopes for approval of a Bitcoin exchange-traded fund got ahead of itself.
The Federal Reserve also announces its interest rate decision on Wednesday and while a hike is priced in, for some investors it’s another reason for caution.
Gloom around China’s economy didn’t help. However, Dogecoin investors ignored these issues and the meme token gained after Twitter owner Elon Musk integrated its
symbol into his Twitter bio on Monday as he rebrands the social media platform into “X”. Elsewhere, the Forkast 500 NFT index continued its slide and U.S. equity
futures traded flat.
""",
"""
Bitcoin is expected to become one of the most promising and high-potential cryptocurrencies in 2023 for a variety of reasons. Firstly, Bitcoin is a popular
cryptocurrency, making it a precious investment. Also, Bitcoin uses state-of-the-art technology, which is very secure. It has a large user ecosystem that supports
its long-term growth. Furthermore, Bitcoin has one of the fastest-growing cryptocurrencies in the world. All these reasons make Bitcoin a cryptocurrency with much
potential, so you should invest in it in 2023.
""",
"""
Bitcoin fell 2.92% over the last 24 hours to US$29,182 as of 07:35 a.m. in Hong Kong for a weekly loss of 3.14%, according to data from CoinMarketCap. The world’s
largest cryptocurrency briefly traded below the US$29,000 mark on Monday evening for the first time in more than a month, or since June 21.
"""
]
articles = [f['description'] for f in get_articles()]
log(str(len(articles)) + " articles: "+str(articles))
summarized_articles = summarize_data(articles, summarizer, tokenizer)
log(str(len(summarized_articles)) + " summarized_articles: "+str(summarized_articles))


['https://www.pap.pl/rss.xml']
10 articles: ['<span>Potrzebujemy więcej osocza - co zrobić, by jego oddawanie weszło nam w krew?</span>\n\n<span><span lang="" xml:lang="">uslusarczyk</span></span>\n\n<span>wt., 07/25/2023 - 08:30</span>\n\n\n        \n    <div class="field field--name-field-image field--type-image field--label-hidden imageWrapper cp">\n                                                          <img alt="Instytut Rozwoju Spraw Społecznych" class="img-responsive" height="3659" src="https://www.pap.pl/sites/default/files/202307/Zdj%C4%99cie_Inf.%20Prasowa_Potrzebujemy%20wi%C4%99cej%20osocza.jpg" width="5489" /><div class="description">Instytut Rozwoju Spraw Społecznych</div>\n                                                            </div>\n\n            <div class="field field--name-field-lead field--type-string-long field--label-hidden field--item">Zapotrzebowanie na osocze i leki produkowane z tego cennego składnika krwi stale rośnie. W wielu przypadkach nie da się ic

In [ ]:
model = BERTopic()
topics, probs = model.fit_transform(summarized_articles)
print(str(len(topics)) + " topics: "+str(topics))


10 topics: [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1]


In [ ]:
topic = model.get_topic_info()
print(str(len(topic)) + " topics: "+str(topic))


1 topics:    Topic  Count                       Name  \
0     -1     10  -1_na_oze_muzeum_narodowe   

                                      Representation  \
0  [na, oze, muzeum, narodowe, energa, polsce, si...   

                                 Representative_Docs  
0  [<span>Cable pooling – niezbędne narzędzie do ...  


In [ ]:
# Run analysis

results = analyse_sentiment(
    summarized_articles,
    pipe
)
log("results: "+str(results))

data analysis took 6.130718469619751 seconds
results: [{'label': 'NEGATIVE', 'score': 0.98665851354599}, {'label': 'NEGATIVE', 'score': 0.893512487411499}, {'label': 'NEGATIVE', 'score': 0.9940681457519531}, {'label': 'NEGATIVE', 'score': 0.993844747543335}, {'label': 'NEGATIVE', 'score': 0.9943663477897644}, {'label': 'NEGATIVE', 'score': 0.9954866766929626}, {'label': 'NEGATIVE', 'score': 0.9804039597511292}, {'label': 'NEGATIVE', 'score': 0.9897624850273132}, {'label': 'NEGATIVE', 'score': 0.9908190369606018}, {'label': 'NEGATIVE', 'score': 0.9934360980987549}]


In [ ]:
inputs = tokenizer.encode("summarize: " + text_data[0], return_tensors='pt',
max_length=512,
truncation=True)
summary_ids = model.generate(inputs, max_length=40, min_length=20, length_penalty=5., num_beams=2)
summary = tokenizer.decode(summary_ids[0])
print(summary)

<pad> bitcoin had a great start to 2023, reversing the downward trend seen throughout the end of 2022. from $16,000 on 1 January, it has risen 63%


In [ ]:
gn = GoogleNews()
s = gn.search('boeing OR airbus')

for entry in s["entries"]:
    print(entry["title"])

NameError: ignored